Extra csv for past fights and events

In [1]:
import config.ConnectionConfig as cc
import pandas as pd

In [2]:

cc.setupEnvironment()
spark = cc.startLocalCluster("UFC_Fighter_Stats")
spark.getActiveSession()

In [13]:
from datetime import date

past_events = spark.read.csv("../data/ufc_event_details.csv", header=True)

#take only the past events

past_events = past_events.toPandas()

past_events.DATE = pd.to_datetime(past_events.DATE, format='%B %d, %Y')
past_events.DATE = past_events.DATE.dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

past_events.head()

past_events.drop('URL', axis=1, inplace=True)

past_events.to_csv("../processed_data/past_events.csv", index=False, sep=",")

past_events

Past fights

In [14]:
from pyspark.sql.functions import split

past_fights = spark.read.csv("../data/ufc_fight_results.csv", header=True)

past_fights.createOrReplaceTempView("fight_results")

past_fights = spark.sql("""
    SELECT 
        EVENT,
        BOUT,
        OUTCOME
    FROM 
        fight_results
""")

#get fighters from bout

past_fights = past_fights.withColumn("fighter1", split(past_fights["BOUT"], " vs. ")[0]) \
    .withColumn("fighter2", split(past_fights["BOUT"], " vs. ")[1])

past_fights = past_fights.drop("BOUT")


past_fights.show()

In [15]:
fighter_details = spark.read.csv("../processed_data/fighter_details.csv", header=True, inferSchema=True)


fighter_details.show()

In [17]:
past_fights.createOrReplaceTempView("past_fights")
fighter_details.createOrReplaceTempView("fighters")

total_df_test = spark.sql("""SELECT
    f.EVENT,
    f.fighter1,
    fr1.fighter_index as fighter1_index,
    f.fighter2,
    fr2.fighter_index as fighter2_index,
    f.OUTCOME
FROM
    past_fights f
JOIN 
    fighters fr1 ON trim(f.fighter1) = trim(fr1.FIGHTER)
JOIN
    fighters fr2 ON trim(f.fighter2) = trim(fr1.FIGHTER)
""")

total_df_test.show()

In [12]:
#join on fighter 1 and 2 from event
past_fights.createOrReplaceTempView("match_results")
fighter_details.createOrReplaceTempView("fighters")


total_df = spark.sql("""SELECT
    f.EVENT,
    f.fighter1,
    fr1.fighter_index as fighter1_index,
    fr1.Height_CM AS height1,
    fr1.Weight_KG AS weight_kg1,
    fr1.AVG_KD AS avg_kd1,
    fr1.AVG_SUB_ATT AS avg_sub_att1,
    fr1.AVG_TD_Percentage AS avg_td_percentage1,
    fr1.AVG_Significant_Strike_Percentage AS avg_significant_strike_percentage1,
    fr1.AVG_TOTAL_STR AS avg_total_str1,
    fr1.AVG_ROUND AS avg_round1,
    fr1.AVG_CTRL_SECONDS AS avg_ctrl_seconds1,
    fr1.total_wins AS total_wins1,
    fr1.total_losses AS total_losses1,
    fr1.total_draws AS total_draws1,
    fr1.Win_Percentage AS win_percentage1,
    fr1.Reach_Conv AS reach_conv1,
    f.fighter2,
    fr2.fighter_index as fighter2_index,
    fr2.Height_CM AS height2,
    fr2.Weight_KG AS weight_kg2,
    fr2.AVG_KD AS avg_kd2,
    fr2.AVG_SUB_ATT AS avg_sub_att2,
    fr2.AVG_TD_Percentage AS avg_td_percentage2,
    fr2.AVG_Significant_Strike_Percentage AS avg_significant_strike_percentage2,
    fr2.AVG_TOTAL_STR AS avg_total_str2,
    fr2.AVG_ROUND AS avg_round2,
    fr2.AVG_CTRL_SECONDS AS avg_ctrl_seconds2,
    fr2.total_wins AS total_wins2,
    fr2.total_losses AS total_losses2,
    fr2.total_draws AS total_draws2,
    fr2.Win_Percentage AS win_percentage2,
    fr2.Reach_Conv AS reach_conv2,
    f.OUTCOME
FROM
    past_fights f
JOIN 
    fighters fr1 ON f.fighter1 = fr1.fighter
JOIN
    fighters fr2 ON f.fighter2 = fr2.fighter;
""")

total_df.show()